In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('../dataset/datagrand_2021_train.csv', sep=',')
train['text'] = train['text'].map(lambda a: a.replace('，', str(30355 - 1)))
train['text'] = train['text'].map(lambda a: a.replace('！', str(30355 - 2)))
train['text'] = train['text'].map(lambda a: a.split(" "))
train['text'] = train['text'].map(lambda a: [int(num) for num in a])
train['1-label'] = train['label'].map(lambda a: int(a.split('-')[0]))
train['2-label'] = train['label'].map(lambda a: int(a.split('-')[1]))

In [3]:
train['label'].nunique()

35

In [31]:
# 统计各类别的样本数
c = train.groupby(['label'], as_index=False)['label'].agg({'cnt': 'count'})
c.sort_values("cnt", inplace=True)
# c.describe()
# rel_label = [0]*36
# for line in c['label']:
#     first,secod = line.split('-')
#     rel_label[int(secod)] = int(first)
# print(rel_label)

In [36]:
# 统计各类别的样本数
c = train.groupby(['2-label'], as_index=False)['2-label'].agg({'cnt': 'count'})
c.sort_values('2-label', ascending=True, inplace=True)
c.describe()
c.loc[c['cnt']<239]

,2-label,cnt
4,5,16
11,12,205
12,13,185
15,16,32
16,17,176
17,18,20
18,19,178
19,20,33
20,21,134
22,23,19


In [6]:
# 统计text的长度信息
train['len'] = train['text'].str.len()
train.describe()

,id,1-label,2-label,len
count,14009.000000,14009.000000,14009.000000,14009.000000
mean,7004.000000,2.993004,13.753159,44.805197
std,4044.194296,2.015951,10.990892,16.104233
min,0.000000,1.000000,1.000000,8.000000
25%,3502.000000,1.000000,4.000000,35.000000
50%,7004.000000,2.000000,9.000000,45.000000
75%,10506.000000,5.000000,25.000000,53.000000
max,14008.000000,10.000000,35.000000,334.000000


In [29]:
# 计算前90%的数据长度
c = train
c.sort_values('len', ascending=True, inplace=True)
c[13809:14009].loc()
train.loc[train['2-label'].isin([5,17,26,23,27])]

,id,text,label,1-label,2-label,len
6410,6410,"[17060, 21114, 4106, 16614, 6180, 648, 8224, 3...",2-17,2,17,15
12736,12736,"[9737, 9283, 25483, 19121, 10647, 8644, 14123,...",10-26,10,26,16
10630,10630,"[21105, 14186, 14684, 3756, 29498, 11259, 1912...",3-5,3,5,17
12875,12875,"[12772, 4198, 30354, 1292, 929, 15439, 2066, 1...",2-17,2,17,18
4360,4360,"[10935, 6832, 22382, 233, 21534, 4198, 25257, ...",8-27,8,27,20
...,...,...,...,...,...,...
2426,2426,"[233, 2103, 16328, 18371, 1396, 233, 8679, 303...",10-26,10,26,175
5215,5215,"[20788, 13783, 16328, 1845, 21655, 2337, 4054,...",10-26,10,26,184
5433,5433,"[2797, 7386, 18239, 26669, 12567, 17281, 2970,...",2-17,2,17,191
13631,13631,"[3110, 6285, 8, 29498, 19910, 23452, 23743, 82...",8-27,8,27,195


In [8]:
#使用正则匹配
import re

m = re.findall('\d+', ',')
max_vob = 0
min_vob = 9999
max_length = 0
for text in train['text']:
    max_length = max(max_length, len(text))
    max_vob = max(max_vob, max(text))
    min_vob = min(min_vob, min(text))
print(max_vob, min_vob, max_length)

30354 8 334


In [9]:
test = pd.read_csv('../dataset/datagrand_2021_test.csv', sep=',')
test['text'] = test['text'].map(lambda a: a.replace('，', str(30355 - 1)))
test['text'] = test['text'].map(lambda a: a.replace('！', str(30355 - 2)))
test['text'] = test['text'].map(lambda a: a.split(" "))
test['text'] = test['text'].map(lambda a: [int(num) for num in a])

In [11]:
# 训练Word Embedding
# skip-gram 对低频词有更好的训练效果
from gensim.models import Word2Vec
import torch

sent1 = [row for row in train['text']]
sent2 = [row for row in test['text']]
word2vec = Word2Vec(sent1 + sent2, min_count=0, vector_size=300, workers=3, window=5, sg=1, epochs=1000)
word2vec.save('word2vec.model')

/home/yxb/anaconda3/envs/nlp/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [16]:
# 训练Word Embedding
# skip-gram 对低频词有更好的训练效果
model = Word2Vec.load('word2vec.model')
model.wv.most_similar(30354, topn=10)

[(2281, 0.42141976952552795),
 (17281, 0.3925333619117737),
 (13535, 0.3719053566455841),
 (29498, 0.3575664162635803),
 (929, 0.32917866110801697),
 (28874, 0.32910993695259094),
 (979, 0.31846386194229126),
 (14547, 0.3110749423503876),
 (25483, 0.3049441874027252),
 (26336, 0.28959640860557556)]

In [89]:
pred_wordEmbedding = torch.tensor(model.wv.vectors)
print(pred_wordEmbedding.shape)
pred_wordEmbedding = torch.cat((torch.zeros(1, 300), pred_wordEmbedding), dim=0)
pred_wordEmbedding

torch.Size([3455, 300])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0750,  0.1899,  0.1656,  ...,  0.1708,  0.0143, -0.0527],
        [ 0.0018,  0.1793, -0.0720,  ..., -0.2750,  0.0836,  0.0457],
        ...,
        [ 0.0006,  0.0835,  0.0598,  ..., -0.0834,  0.0440,  0.0233],
        [-0.0281,  0.0595,  0.0540,  ..., -0.0322,  0.0387, -0.0218],
        [ 0.0507,  0.1106, -0.0527,  ..., -0.0159, -0.0158,  0.0163]])

In [56]:
import re

m = re.findall('\d+', ',')
max_vob = 0
min_vob = 9999
max_length = 0
for text in test['text']:
    vobs = text.split(' ')
    max_length = max(max_length, len(vobs))
    vobs = [int(vob) for vob in vobs if len(re.findall('\d+', vob)) > 0]
    max_vob = max(max_vob, max(vobs))
    min_vob = min(min_vob, min(vobs))
print(max_vob, min_vob, max_length)

AttributeError: 'list' object has no attribute 'split'

In [9]:
import torch

a = torch.randn((4, 5))
torch.max(a, 1)

torch.return_types.max(
values=tensor([0.1448, 0.9003, 0.6007, 1.3291]),
indices=tensor([4, 1, 3, 0]))

In [10]:
submit = pd.read_csv('../dataset/sample_submission.csv', sep=',')
submit['label'][0]
mysubmit = pd.read_csv('../dataset/submit.csv', sep=',')
mysubmit['label'][0]

'2-6'

In [11]:
d = mysubmit.groupby(['label'], as_index=False)['label'].agg({'cnt': 'count'})
d.sort_values("label", inplace=True)
d

,label,cnt
0,1-1,451
1,1-10,304
2,1-4,392
3,1-9,1013
4,2-11,33
5,2-14,65
6,2-2,401
7,2-25,113
8,2-3,448
9,2-33,282


In [12]:
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer(vocab_file='../dataset/vocab.txt')
inputs = tokenizer(
    "10988 8346 9955 27506 1845 29179 28438 10300 27321 9755 ， 27321 14873 29731 487 22191 10449 2617 1433 ， 24965 6527 26669 12567 17281 1506 979 13535 27321 8346",
    truncation=True, return_tensors='pt', max_length=512)
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]])}

In [13]:
rand = torch.rand(inputs.input_ids.shape)
mask_arr = rand < 0.15
mask_arr

tensor([[False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False]])

In [14]:
# 不mask cls 和 seq
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102)
mask_arr

tensor([[False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False]])

In [15]:
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

/home/yxb/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  """Entry point for launching an IPython kernel.


[5, 20]

In [16]:
inputs.input_ids[0, selection] = 103
inputs

{'input_ids': tensor([[  101, 11098,  8456, 10065, 27616,   103, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           103, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  101, 11098,  8456, 10065, 27616,  1955, 29289, 28548, 10410, 27431,
          9865,   118, 27431, 14983, 29841,   597, 22301, 10559,  2727,  1543,
           118, 25075,  6637, 26779, 12677, 17391,  1616,  1089, 13645, 27431,
          8456,   102]])}